In [1]:
import pandas as pd
import joblib
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def add_labels(df, future_periods=3, threshold=0.005):
    df = df.copy()
    df['future_close'] = df['close'].shift(-future_periods)
    df['future_return'] = (df['future_close'] - df['close']) / df['close']
    df['label'] = (df['future_return'] > threshold).astype(int)
    return df

input_folder = r"C:\Users\praga\OneDrive\Desktop\Trading Bot\data\signals"
model_folder = r"C:\Users\praga\OneDrive\Desktop\Trading Bot\ml_model"
os.makedirs(model_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith("_signals.csv"):
        symbol = file.replace("_signals.csv", "")
        print(f"🔄 Training model for {symbol}...")

        df = pd.read_csv(os.path.join(input_folder, file))
        df = add_labels(df)
        df = df.dropna()

        if df['label'].nunique() < 2:
            print(f"⚠️ Skipping {symbol}: Not enough label variety (only {df['label'].unique()})")
            continue

        features = ['rsi14', 'ema20', 'MACD_12_26_9', 'MACDs_12_26_9']
        X = df[features]
        y = df['label']

        if len(X) < 10:
            print(f"⚠️ Skipping {symbol}: Not enough data ({len(X)} rows)")
            continue

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
        model.fit(X_train, y_train)

        print(classification_report(y_test, model.predict(X_test)))

        model_path = os.path.join(model_folder, f"{symbol}_model.pkl")
        joblib.dump(model, model_path)
        print(f"✅ Model saved for {symbol} → {model_path}\n")


🔄 Training model for avaxusdt...
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5

✅ Model saved for avaxusdt → C:\Users\praga\OneDrive\Desktop\Trading Bot\ml_model\avaxusdt_model.pkl

🔄 Training model for bnbusdt...
⚠️ Skipping bnbusdt: Not enough label variety (only [0])
🔄 Training model for btcusdt...
⚠️ Skipping btcusdt: Not enough label variety (only [0])
🔄 Training model for ethusdt...
⚠️ Skipping ethusdt: Not enough label variety (only [0])
🔄 Training model for solusdt...
⚠️ Skipping solusdt: Not enough label variety (only [0])
🔄 Training model for suiusdt...
⚠️ Skipping suiusdt: Not enough label variety (only [0])
🔄 Training model for tonusdt...
⚠️ Skipping tonusdt: Not enough label variety (only [0])
🔄 Training model for trxusdt...
⚠️ Skipping trxusdt: Not e